<a href="https://colab.research.google.com/github/leejj0924/DACON_PROJECT_1/blob/main/DACON_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
from pandas.plotting import scatter_matrix
import seaborn as sns

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgbm
import xgboost as xgb

In [10]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SequentialFeatureSelector

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import RocCurveDisplay


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

from sklearn.ensemble import RandomForestClassifier

In [13]:
df_trn = pd.read_csv('/content/DACON_PROJECT_-1/train.csv')
df_tst = pd.read_csv('/content/DACON_PROJECT_-1/test.csv')

In [14]:
df_trn['attack_type'].value_counts()


,count
attack_type,
Benign,8791
Hulk,1719
Port_Scanning,793
DDoS,471
FTP_Brute_Force,47
GoldenEye,41
Slow_HTTP,34
SSH_Brute_Force,30
Botnet,27


In [15]:
df_trn.isna().sum()

,0
ID,0
ip_src,2512
port_src,3374
ip_dst,1304
port_dst,2287
protocol,0
duration,1074
pkt_count_fwd,0
pkt_count_bwd,0
rate_fwd_pkts,1171


In [16]:
import ipaddress

def is_private_ip(ip):
  try:
      return int(ipaddress.ip_address(ip).is_private)
  except:
      return None

def process_ip_column(df, col):
  df[f'{col}_private'] = df[col].apply(is_private_ip)

  mode = df[f'{col}_private'].mode()
  if not mode.empty:
    df[f'{col}_private'] = df[f'{col}_private'].fillna(mode[0])
  else:
    df[f'{col}_private'] = df[f'{col}_private'].fillna(0)
  return df

df_trn = process_ip_column(df_trn, 'ip_src')
df_trn = process_ip_column(df_trn, 'ip_dst')
df_trn = df_trn.drop(columns=['ip_src', 'ip_dst'])

In [17]:
df_trn.head()

,ID,port_src,port_dst,protocol,duration,pkt_count_fwd,pkt_count_bwd,rate_fwd_pkts,rate_bwd_pkts,rate_fwd_bytes,...,payload_bwd_mean,tcp_win_fwd_init,tcp_win_bwd_init,tcp_syn_count,tcp_psh_count,tcp_rst_count,iat_avg_packets,attack_type,ip_src_private,ip_dst_private
0,TRAIN_00000,3721.0,55.0,UDP,0.000231,2,2,8656.974200,8656.974200,259709.226006,...,81.000000,0,0,0,0,0,NaN,Benign,1.0,1.0
1,TRAIN_00001,NaN,91.0,TCP,0.000000,0,1,0.000000,0.000000,0.000000,...,0.000000,0,16392,0,0,0,1.499097e+09,Benign,1.0,1.0
2,TRAIN_00002,NaN,83.0,TCP,0.606002,11,5,18.151760,8.250800,1727.717487,...,790.125000,29200,28960,2,4,1,4.040012e-02,Hulk,1.0,1.0
3,TRAIN_00003,47668.0,NaN,TCP,1.003829,6,6,5.977114,5.977114,324.756507,...,993.416667,29200,28960,3,2,0,9.125718e-02,Hulk,1.0,1.0
4,TRAIN_00004,51753.0,451.0,TCP,181.195271,62,89,NaN,0.491183,6.573019,...,NaN,8192,29200,2,22,0,NaN,Benign,1.0,0.0


In [18]:
from sklearn.impute import KNNImputer

In [19]:
numeric_cols = df_trn.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_cols = [col for col in numeric_cols if col != 'attack_type']
data_for_impute = df_trn[numeric_cols]
imputer = KNNImputer(n_neighbors=5)  # 유사한 5개 이웃 기준 평균
imputed_data = imputer.fit_transform(data_for_impute)
imputed_df = pd.DataFrame(imputed_data, columns=numeric_cols)
df_trn[numeric_cols] = imputed_df

In [20]:
df_trn = df_trn.drop(columns=['ID'])

attack_label_encoder = LabelEncoder()
protocol_label_encoder = LabelEncoder()
df_trn['attack_type'] = attack_label_encoder.fit_transform(df_trn['attack_type'])
df_trn['protocol'] = protocol_label_encoder.fit_transform(df_trn['protocol'])

y = df_trn['attack_type']
X = df_trn.drop(columns=['attack_type'])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_val, X_final_test, y_train_val, y_final_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.2, random_state=42
)

print("데이터 분할 결과:")
print(f"학습 데이터:     {X_train.shape[0]} samples ({X_train.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"검증 데이터:     {X_val.shape[0]} samples ({X_val.shape[0]/len(X_scaled)*100:.1f}%)")
print(f"최종 테스트:     {X_final_test.shape[0]} samples ({X_final_test.shape[0]/len(X_scaled)*100:.1f}%)")

train_class_counts = y_train.value_counts().sort_index()
print(f"\n학습 데이터 클래스 분포:")
print(train_class_counts)

minority_classes = train_class_counts[train_class_counts <= 100].index
print(f"\n100개 이하 클래스: {minority_classes.tolist()}")
print(f"해당 클래스 개수: {len(minority_classes)}개")

데이터 분할 결과:
학습 데이터:     7679 samples (64.0%)
검증 데이터:     1920 samples (16.0%)
최종 테스트:     2400 samples (20.0%)

학습 데이터 클래스 분포:
attack_type
0     5581
1       15
2      307
3       33
4       28
5     1122
6      525
7       15
8       20
9       18
10      11
11       4
Name: count, dtype: int64

100개 이하 클래스: [1, 3, 4, 7, 8, 9, 10, 11]
해당 클래스 개수: 8개


In [12]:
pip install imblearn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.4/238.4 kB 6.1 MB/s eta 0:00:00


In [21]:
from imblearn.over_sampling import SMOTE
from collections import Counter

print("="*60)
print("소수 클래스 보수적 증강 (SMOTE)")
print("="*60)

original_distribution = Counter(y_train)
print("원본 분포:")
for class_id, count in sorted(original_distribution.items()):
    print(f"  클래스 {class_id}: {count}개")

# 타겟 분포 계산 (100개 이하 클래스만 3배 증강)
target_distribution = {}
for class_id, count in original_distribution.items():
    if class_id in minority_classes and count <= 100:
        target_distribution[class_id] = min(count * 3, 300)  # 최대 300개로 제한
    else:
        target_distribution[class_id] = count

print(f"\n타겟 분포 (100개 이하 클래스 3배 증강):")
for class_id, count in sorted(target_distribution.items()):
    original_count = original_distribution[class_id]
    if count > original_count:
        print(f"  클래스 {class_id}: {original_count} → {count}개 (+{count-original_count})")
    else:
        print(f"  클래스 {class_id}: {count}개")

# SMOTE 적용
smote = SMOTE(
    sampling_strategy=target_distribution,
    random_state=42,
    k_neighbors=3  # 보수적 설정
)

X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# 결과 확인
final_distribution = Counter(y_train_balanced)
print(f"\n증강 후 분포:")
total_added = 0
for class_id, count in sorted(final_distribution.items()):
    original_count = original_distribution[class_id]
    added = count - original_count
    total_added += added
    if added > 0:
        print(f"  클래스 {class_id}: {count}개 (+{added})")
    else:
        print(f"  클래스 {class_id}: {count}개")

print(f"\n총 증강된 샘플: {total_added}개")
print(f"최종 학습 데이터: {len(X_train_balanced)}개")

소수 클래스 보수적 증강 (SMOTE)
원본 분포:
  클래스 0: 5581개
  클래스 1: 15개
  클래스 2: 307개
  클래스 3: 33개
  클래스 4: 28개
  클래스 5: 1122개
  클래스 6: 525개
  클래스 7: 15개
  클래스 8: 20개
  클래스 9: 18개
  클래스 10: 11개
  클래스 11: 4개

타겟 분포 (100개 이하 클래스 3배 증강):
  클래스 0: 5581개
  클래스 1: 15 → 45개 (+30)
  클래스 2: 307개
  클래스 3: 33 → 99개 (+66)
  클래스 4: 28 → 84개 (+56)
  클래스 5: 1122개
  클래스 6: 525개
  클래스 7: 15 → 45개 (+30)
  클래스 8: 20 → 60개 (+40)
  클래스 9: 18 → 54개 (+36)
  클래스 10: 11 → 33개 (+22)
  클래스 11: 4 → 12개 (+8)

증강 후 분포:
  클래스 0: 5581개
  클래스 1: 45개 (+30)
  클래스 2: 307개
  클래스 3: 99개 (+66)
  클래스 4: 84개 (+56)
  클래스 5: 1122개
  클래스 6: 525개
  클래스 7: 45개 (+30)
  클래스 8: 60개 (+40)
  클래스 9: 54개 (+36)
  클래스 10: 33개 (+22)
  클래스 11: 12개 (+8)

총 증강된 샘플: 288개
최종 학습 데이터: 7967개


In [22]:
from sklearn.utils.class_weight import compute_class_weight

In [23]:
# 증강된 데이터 기준으로 클래스 가중치 계산
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_balanced),
    y=y_train_balanced
)

class_weight_dict = dict(enumerate(class_weights))
print("클래스 가중치 (증강 후):")
for class_id, weight in sorted(class_weight_dict.items()):
    print(f"  클래스 {class_id}: {weight:.4f}")

# 샘플 가중치 생성
sample_weights = pd.Series(y_train_balanced).map(class_weight_dict)
n_classes = len(np.unique(y_train_balanced))

클래스 가중치 (증강 후):
  클래스 0: 0.1190
  클래스 1: 14.7537
  클래스 2: 2.1626
  클래스 3: 6.7062
  클래스 4: 7.9038
  클래스 5: 0.5917
  클래스 6: 1.2646
  클래스 7: 14.7537
  클래스 8: 11.0653
  클래스 9: 12.2948
  클래스 10: 20.1187
  클래스 11: 55.3264


In [24]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, accuracy_score

print("="*60)
print("Individual Model Training (증강된 데이터)")
print("="*60)

# XGBoost
print("\nTraining XGBoost...")
xgb_model = XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    objective='multi:softprob',
    num_class=n_classes,
    eval_metric='mlogloss',
    use_label_encoder=False,
    random_state=42,
    verbosity=0
)
xgb_model.fit(X_train_balanced, y_train_balanced, sample_weight=sample_weights)

# LightGBM
print("Training LightGBM...")
lgb_model = LGBMClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    objective='multiclass',
    num_class=n_classes,
    metric='multi_logloss',
    random_state=42,
    verbosity=-1,
    force_col_wise=True
)
lgb_model.fit(X_train_balanced, y_train_balanced, sample_weight=sample_weights)

# CatBoost
print("Training CatBoost...")
cat_model = CatBoostClassifier(
    iterations=300,
    depth=6,
    learning_rate=0.1,
    loss_function='MultiClass',
    random_seed=42,
    verbose=False
)
cat_model.fit(X_train_balanced, y_train_balanced, sample_weight=sample_weights)

print("All models trained successfully!")

Individual Model Training (증강된 데이터)

Training XGBoost...
Training LightGBM...
Training CatBoost...
All models trained successfully!


In [25]:
print("\n" + "="*60)
print("Validation Set Performance")
print("="*60)

# 개별 모델 예측
xgb_val_preds = xgb_model.predict(X_val)
lgb_val_preds = lgb_model.predict(X_val)
cat_val_preds = cat_model.predict(X_val)

# 1차원 배열로 변환
xgb_val_preds = np.array(xgb_val_preds).flatten()
lgb_val_preds = np.array(lgb_val_preds).flatten()
cat_val_preds = np.array(cat_val_preds).flatten()

# 개별 모델 성능 계산
models = ['XGBoost', 'LightGBM', 'CatBoost']
predictions = [xgb_val_preds, lgb_val_preds, cat_val_preds]

individual_scores = []
print("개별 모델 성능:")
for model_name, preds in zip(models, predictions):
    acc = accuracy_score(y_val, preds)
    f1_macro = f1_score(y_val, preds, average='macro')
    f1_weighted = f1_score(y_val, preds, average='weighted')

    individual_scores.append(f1_macro)  # Macro F1을 기준으로 가중치 계산
    print(f"{model_name:15} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")

# 앙상블 방법들
print(f"\n앙상블 성능:")

# 1. 단순 평균 (Hard Voting)
simple_ensemble = []
for i in range(len(xgb_val_preds)):
    votes = [xgb_val_preds[i], lgb_val_preds[i], cat_val_preds[i]]
    # 다수결 투표
    simple_ensemble.append(max(set(votes), key=votes.count))
simple_ensemble = np.array(simple_ensemble)

# 2. 성능 기반 가중 평균
total_score = sum(individual_scores)
weights = [score/total_score for score in individual_scores]
print(f"성능 기반 가중치: XGB={weights[0]:.3f}, LGB={weights[1]:.3f}, CAT={weights[2]:.3f}")

weighted_ensemble = []
for i in range(len(xgb_val_preds)):
    weighted_vote = (weights[0] * xgb_val_preds[i] +
                    weights[1] * lgb_val_preds[i] +
                    weights[2] * cat_val_preds[i])
    weighted_ensemble.append(int(round(weighted_vote)))
weighted_ensemble = np.array(weighted_ensemble)

# 3. 확률 기반 소프트 보팅
xgb_val_proba = xgb_model.predict_proba(X_val)
lgb_val_proba = lgb_model.predict_proba(X_val)
cat_val_proba = cat_model.predict_proba(X_val)

# 확률 평균
avg_proba = (xgb_val_proba + lgb_val_proba + cat_val_proba) / 3
soft_ensemble = np.argmax(avg_proba, axis=1)

# 앙상블 성능 평가
ensemble_methods = ['Hard Voting', 'Weighted (Performance)', 'Soft Voting']
ensemble_preds = [simple_ensemble, weighted_ensemble, soft_ensemble]

val_results = []
for method_name, preds in zip(ensemble_methods, ensemble_preds):
    acc = accuracy_score(y_val, preds)
    f1_macro = f1_score(y_val, preds, average='macro')
    f1_weighted = f1_score(y_val, preds, average='weighted')

    val_results.append({
        'Model': method_name,
        'Accuracy': acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

    print(f"{method_name:20} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")

# 최고 성능 앙상블 선택
best_ensemble_idx = np.argmax([r['Macro F1'] for r in val_results])
best_ensemble_name = ensemble_methods[best_ensemble_idx]
best_ensemble_preds = ensemble_preds[best_ensemble_idx]

print(f"\n최고 성능 앙상블: {best_ensemble_name}")
print(f"Best Validation Macro F1: {val_results[best_ensemble_idx]['Macro F1']:.4f}")


Validation Set Performance
개별 모델 성능:
XGBoost         | Accuracy: 0.9865 | Macro F1: 0.6880 | Weighted F1: 0.9873
LightGBM        | Accuracy: 0.9896 | Macro F1: 0.7368 | Weighted F1: 0.9902
CatBoost        | Accuracy: 0.9672 | Macro F1: 0.6264 | Weighted F1: 0.9732

앙상블 성능:
성능 기반 가중치: XGB=0.335, LGB=0.359, CAT=0.305
Hard Voting          | Accuracy: 0.9885 | Macro F1: 0.7273 | Weighted F1: 0.9891
Weighted (Performance) | Accuracy: 0.9641 | Macro F1: 0.6180 | Weighted F1: 0.9697
Soft Voting          | Accuracy: 0.9896 | Macro F1: 0.7388 | Weighted F1: 0.9902

최고 성능 앙상블: Soft Voting
Best Validation Macro F1: 0.7388


In [26]:
print("\n" + "="*60)
print("Final Test Set Performance")
print("="*60)

# 개별 모델 예측
xgb_test_preds = xgb_model.predict(X_final_test)
lgb_test_preds = lgb_model.predict(X_final_test)
cat_test_preds = cat_model.predict(X_final_test)

# 1차원 배열로 변환
xgb_test_preds = np.array(xgb_test_preds).flatten()
lgb_test_preds = np.array(lgb_test_preds).flatten()
cat_test_preds = np.array(cat_test_preds).flatten()

# 개별 모델 성능
print("개별 모델 성능:")
individual_test_results = []
for model_name, preds in zip(models, [xgb_test_preds, lgb_test_preds, cat_test_preds]):
    acc = accuracy_score(y_final_test, preds)
    f1_macro = f1_score(y_final_test, preds, average='macro')
    f1_weighted = f1_score(y_final_test, preds, average='weighted')

    individual_test_results.append({
        'Model': model_name,
        'Accuracy': acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

    print(f"{model_name:15} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")

# 앙상블 예측 (검증에서 가장 좋았던 방법 사용)
print(f"\n앙상블 성능:")

# 1. Hard Voting
simple_test_ensemble = []
for i in range(len(xgb_test_preds)):
    votes = [xgb_test_preds[i], lgb_test_preds[i], cat_test_preds[i]]
    simple_test_ensemble.append(max(set(votes), key=votes.count))
simple_test_ensemble = np.array(simple_test_ensemble)

# 2. 성능 기반 가중 평균 (검증 성능 기준 가중치 재사용)
weighted_test_ensemble = []
for i in range(len(xgb_test_preds)):
    weighted_vote = (weights[0] * xgb_test_preds[i] +
                    weights[1] * lgb_test_preds[i] +
                    weights[2] * cat_test_preds[i])
    weighted_test_ensemble.append(int(round(weighted_vote)))
weighted_test_ensemble = np.array(weighted_test_ensemble)

# 3. 소프트 보팅
xgb_test_proba = xgb_model.predict_proba(X_final_test)
lgb_test_proba = lgb_model.predict_proba(X_final_test)
cat_test_proba = cat_model.predict_proba(X_final_test)

avg_test_proba = (xgb_test_proba + lgb_test_proba + cat_test_proba) / 3
soft_test_ensemble = np.argmax(avg_test_proba, axis=1)

# 앙상블 성능 평가
test_ensemble_preds = [simple_test_ensemble, weighted_test_ensemble, soft_test_ensemble]

test_results = []
for method_name, preds in zip(ensemble_methods, test_ensemble_preds):
    acc = accuracy_score(y_final_test, preds)
    f1_macro = f1_score(y_final_test, preds, average='macro')
    f1_weighted = f1_score(y_final_test, preds, average='weighted')

    test_results.append({
        'Model': method_name,
        'Accuracy': acc,
        'Macro F1': f1_macro,
        'Weighted F1': f1_weighted
    })

    print(f"{method_name:20} | Accuracy: {acc:.4f} | Macro F1: {f1_macro:.4f} | Weighted F1: {f1_weighted:.4f}")

# 최고 성능
best_test_f1 = max([r['Macro F1'] for r in test_results])
best_individual_f1 = max([r['Macro F1'] for r in individual_test_results])

print(f"\nBest Individual Test F1: {best_individual_f1:.4f}")
print(f"Best Ensemble Test F1: {best_test_f1:.4f}")

if best_test_f1 > best_individual_f1:
    improvement = best_test_f1 - best_individual_f1
    print(f"앙상블이 개별 모델보다 {improvement:.4f} 향상!")
else:
    print("앙상블이 개별 모델보다 성능 향상이 없음")


Final Test Set Performance
개별 모델 성능:
XGBoost         | Accuracy: 0.9892 | Macro F1: 0.8010 | Weighted F1: 0.9896
LightGBM        | Accuracy: 0.9925 | Macro F1: 0.8451 | Weighted F1: 0.9927
CatBoost        | Accuracy: 0.9767 | Macro F1: 0.7275 | Weighted F1: 0.9796

앙상블 성능:
Hard Voting          | Accuracy: 0.9917 | Macro F1: 0.8349 | Weighted F1: 0.9919
Weighted (Performance) | Accuracy: 0.9746 | Macro F1: 0.7592 | Weighted F1: 0.9774
Soft Voting          | Accuracy: 0.9917 | Macro F1: 0.8351 | Weighted F1: 0.9919

Best Individual Test F1: 0.8451
Best Ensemble Test F1: 0.8351
앙상블이 개별 모델보다 성능 향상이 없음


In [27]:
# 2. IP 처리
df_tst = process_ip_column(df_tst, 'ip_src')
df_tst = process_ip_column(df_tst, 'ip_dst')
df_tst = df_tst.drop(columns=['ip_src', 'ip_dst'])

# 3. KNN 결측치 대체
test_numeric_cols = df_tst.select_dtypes(include=['float64', 'int64']).columns.tolist()
test_numeric_cols = [col for col in test_numeric_cols if col != 'attack_type'] if 'attack_type' in df_tst.columns else test_numeric_cols

test_data_for_impute = df_tst[test_numeric_cols]
test_imputed_data = imputer.transform(test_data_for_impute)
test_imputed_df = pd.DataFrame(test_imputed_data, columns=test_numeric_cols)
df_tst[test_numeric_cols] = test_imputed_df

# 4. 'ID' 제거 (있다면)
if 'ID' in df_tst.columns:
    df_tst = df_tst.drop(columns=['ID'])

# 5. protocol 라벨 인코딩
df_tst['protocol'] = protocol_label_encoder.transform(df_tst['protocol'])

# 6. attack_type 제거 (있다면)
if 'attack_type' in df_tst.columns:
    df_tst = df_tst.drop(columns=['attack_type'])

# 7. 스케일링
X_test_scaled = scaler.transform(df_tst)

In [28]:
from scipy.special import softmax
from scipy.stats import mode
from sklearn.preprocessing import OneHotEncoder

# soft voting을 위한 확률 예측
test_preds_proba = np.zeros((X_test_scaled.shape[0], len(np.unique(y))))

ensemble_models = [xgb_model, lgb_model, cat_model]
n_models = len(ensemble_models)

for model in ensemble_models:
    if hasattr(model, 'predict_proba'):
        proba = model.predict_proba(X_test_scaled)
        test_preds_proba += proba
    else:
        print(f"Warning: Model {type(model).__name__} does not support predict_proba and is skipped for soft voting.")

# 평균 확률로 클래스 선택
if n_models > 0:
    test_preds_final = np.argmax(test_preds_proba / n_models, axis=1)
else:
    # Handle case where no models could contribute probabilities
    print("Error: No models were available for soft voting probability prediction.")
    # You might want to raise an error or handle this case appropriately
    # For now, setting to a placeholder or first class
    test_preds_final = np.zeros(X_test_scaled.shape[0], dtype=int)

# 라벨 디코딩
test_pred_labels = attack_label_encoder.inverse_transform(test_preds_final)

In [ ]:
'''
sample_submission = pd.read_csv('submission_example.csv')

sample_submission['attack_type'] = test_pred_labels

filename = 'base_submission.csv'
sample_submission.to_csv(filename, index=False)
'''